In [ ]:
pip install kiwipiepy

In [26]:
# 1. 필요한 라이브러리 임포트
import json
import re
from kiwipiepy import Kiwi
from tqdm import tqdm
from pprint import pprint as pp

# JSON 파일 로딩
with open("/content/이준석.json", "r", encoding="utf-8") as f:
    data = json.load(f)

articles = data["이준석"]
contents = [article["content"] for article in articles]

In [19]:
pp(type(data))

<class 'dict'>


In [20]:
pp(data.keys())

dict_keys(['이준석'])


In [21]:
for i in data.keys():
  pp(f"key : {i}; type: { type(data[i])}; lenghth: {len(data[i])}")

"key : 이준석; type: <class 'list'>; lenghth: 1688"


In [27]:
import re

def clean_text(text):
    text = re.sub(r'<.*?>', '', text)                 # HTML 태그 제거
    text = re.sub(r'\n+', ' ', text)                  # 줄바꿈 -> 공백
    text = re.sub(r'["“”‘’\'!?…·,;:\(\)\[\]\{\}]', '', text)  # 주요 문장부호 제거
    text = re.sub(r'[^\w\s가-힣]', '', text)           # 그 외 특수문자 제거 (이중 필터링)
    text = re.sub(r'\s+', ' ', text).strip()          # 다중 공백 제거
    return text

# 적용
cleaned_contents = [clean_text(text) for text in contents]



In [31]:
# 4. 문장 단위로 품사 태깅 결과를 병합
rows = []
for doc_id, text in enumerate(cleaned_contents):
    sentences = kiwi.split_into_sents(text)
    for sent in sentences:
        analyzed = kiwi.analyze(sent.text)[0][0]
        tagged_str = ' '.join([f"{token.form}({token.tag})" for token in analyzed])
        rows.append({
            "doc_id": doc_id,
            "original_sentence": sent.text,
            "tagged_sentence": tagged_str
        })

# 5. DataFrame 생성
df_tagged = pd.DataFrame(rows)


In [29]:
import pandas as pd

rows = []

for article in tagged_articles:
    for sent in article:
        for token, tag in sent["tagged_tokens"]:
            rows.append({
                "doc_id": sent["doc_id"],
                "sentence": sent["sentence"],
                "token": token,
                "tag": tag
            })

df = pd.DataFrame(rows)


In [32]:
df_tagged.to_csv("이준석_문장별_품사태깅.csv", index=False, encoding="utf-8-sig")
